# **备注:此文档第一章\第二章,我第一遍整理完并没有理解**


# 装饰器是什么?

装饰器是程序开发中经常用到的一个功能,用好了装饰器,开发效率如虎添翼,

所以这也是Python面试中必问的问题.

但对于好多初次接触这个知识的人来讲,这个功能有点绕,自学时直接绕过去,

然后面试问道了就挂了,因为装饰器是程序开发的基础知识,

这个都不会,别跟人家说你会Python
看下面的内容,保证我要学会装饰器.

## 函数重名的问题

In [1]:
def test1():
    print('----1----')

def test1():
    print('----2----')

test1()  

#这里打印 ----2---- 因为全部加载了缓存后,test1现在指向了下面为2的这个函数

----2----


> ###第一波###


In [2]:
def foo():
    print('foo')

foo     #表示是函数
foo()   #表示执行函数

foo


> ###第二波###

In [11]:
def foo():
    print('foo')

foo = lambda x:x+1  #这个运行时候报错,因为没定义X

foo() #执行下面的lambda表达式,而不在是原来的foo函数,因为foo这个名字被重新指向了另外一个匿名数

foo


In [15]:
def w1(func):
    def inner():
        print('----正在验证----')
        func()
    return inner

def f1():
    print('----f1----')

def f2():
    print('----f2----')

f1 = w1(f1) #把f1传递给func func指向了f1原来的那个函数 w1反回inner这个函数的引用 f1指向了这个引用
f1() #f1指向了inner这个函数的地址 f1()现在调用的就是inner这个函数

----正在验证----
----f1----


## 上述案例如果加上装饰器

In [22]:
def w1(func):
    def inner():
        print('----正在验证----')
        if True:
            func()
        else:
            print("没有权限")
    return inner

@w1  #这里的@w1 <==> f1 = w1(f1)
def f1():
    print('----f1----')

@w1
def f2():
    print('----f2----')


#f1 = w1(f1) #把f1传递给func func指向了f1原来的那个函数 w1反回inner这个函数的引用 f1指向了这个引用

#f1() #f1指向了inner这个函数的地址 f1()现在调用的就是inner这个函数

f1()
f2()


----正在验证----
----f1----
----正在验证----
----f2----


## 多个装饰器
有点儿晕,为什么是这么个顺序呢

In [34]:
#定义函数:完成包裹数据
def makeBold(fn):
    def wrapped():
        print('----1----')
        return "<b>"+fn()+"</b>"
    return wrapped


#定义函数:完成包裹数据
def makeItalic(fn):
    def wrapped():
        print("----2----")
        return "<i>"+fn()+"</i>"
    return wrapped

@makeBold #@makeBold <====> test3 = makeBold(test3)   再封装这个  因为它下面没有函数调用
@makeBold
@makeItalic #@makeItalic <===> test3 = makeItalic(test3)  先封装这个
def test3():#这个画图就知道了
    print("----3----")
    return "hello world-3"

result = test3()
print(result)

----1----
----1----
----2----
----3----
<b><b><i>hello world-3</i></b></b>


# 装饰器什么时候进行装饰
## 单个装饰器

In [35]:
def w1(func):
    def inner():
        print('----正在验证----')
        func()
    return inner

#只要python解释器执行到了这个代码(@w1),那么就会自动的进行装饰,而不是等到调用的时候才装饰的
@w1     # @w1 <==> f1=w1(f1)
def f1():
    print('----f1----')

#在调用f1之前,已经进行装饰了
f1() 

----正在验证----
----f1----


## 多个装饰器

In [36]:
def w1(func):
    print("----正在验证1----")
    def inner():
        print("----正在获取权限1----")
        func()
    return inner

def w2(func):
    print("----正在验证2----")
    def inner():
        print("----正在获取权限2----")
        func()
    return inner


# 只要python解释器执行到了这个代码,那么就会自动的进行装饰,而不是等到调用的时候才装饰
@w1
@w2     #装饰是从下往上,调用是从上往下  执行结果为 ----正在验证2----,--正在1---,---获取1--,---获取2---,---f1---
def f1():
    print("----f1----")

#在调用f1之前,已经进行装饰了
f1()

----正在验证2----
----正在验证1----
----正在获取权限1----
----正在获取权限2----
----f1----


## 使用装饰器对无参数的函数进行装饰

In [37]:
def func(functionName):
    print("----func----1----")
    def func_in():
        print("----func_in----1----")
        functionName()
        print("----func_in----2----")

    print("----func----2----")
    return func_in

@func
def test():
    print("----test----")

#test = func(test)

test()

----func----1----
----func----2----
----func_in----1----
----test----
----func_in----2----


## 使用装饰器对有参数的函数进行装饰

In [49]:
def func(functionName):
    print("----func----1----")
    def func_in(a,b): #如果a,b 没有定义,那么导致17行的调用个失败
        print("----func_in----1----")
        functionName(a,b) #如果a,b 没有当做实参传递,那么会导致调用12行的的函数失败
        print("----func_in----2----")

    print("----func----2----")
    return func_in

@func
def test(a,b):
    print("----test-a=%d-b=%d--"%(a,b))

# test = func(test)

test(11,22)

----func----1----
----func----2----
----func_in----1----
----test-a=11-b=22--
----func_in----2----


## 使用装饰器对不定长参数的函数进行装饰

In [50]:
def func(functionName):
    print("----func----1----")
    def func_in(*args,**kwargs): #如果*args,**kwargs 没有定义,那么导致19,21行的调用个失败
        print("----func_in----1----")
        functionName(*args,**kwargs) #如果args,kwargs会报错,因为没有解包  *args,**kwargs这是解包
        print("----func_in----2----")#没有当做实参传递,那么会导致调用12,16行的的函数失败

    print("----func----2----")
    return func_in

@func
def test(a,b,c):
    print("----test-a=%d-b=%d-c=%d-"%(a,b,c))

@func
def test2(a,b,c,d):
    print("----test-a=%d-b=%d-c=%d-d=%d"%(a,b,c,d))

test(11,22,33)
print('')

test2(44,55,66,77)

----func----1----
----func----2----
----func----1----
----func----2----
----func_in----1----
----test-a=11-b=22-c=33-
----func_in----2----

----func_in----1----
----test-a=44-b=55-c=66-d=77
----func_in----2----


## 使用装饰器对有返回值的函数进行装饰

In [52]:
def func(functionName):
    print("----func----1----")
    def func_in():
        print("----func_in----1----")
        ret = functionName() #保存反回来的haha
        print("----func_in----2----")
        return ret #把haha反回到16行的调用
    print("----func----2----")
    return func_in

@func
def test():
    print("----test----")
    return "haha"

ret = test()
print("test return value is %s"%(ret))


----func----1----
----func----2----
----func_in----1----
----test----
----func_in----2----
test return value is haha


## 使用通用装饰器完成对函数进行装饰

In [54]:
def func(functionName):
    def func_in(*args,**kwargs):
        print("----正在记录中----")
        ret = functionName(*args,**kwargs)
        return ret
    return func_in

@func
def test():
    print("----test----")
    return "haha"

@func
def test2():
    print("----test2----")

@func
def test3(a,b):
    print("----test3-a=%d-b=%d----"%(a,b))


ret = test()
print("test return value is %s"%(ret))
print('')

ret = test2()
print("test return value is %s"%(ret))
print('')

test3(11,22)

----正在记录中----
----test----
test return value is haha

----正在记录中----
----test2----
test return value is None

----正在记录中----
----test3-a=11-b=22----


## 带有参数的装饰器

In [45]:
def func_args(args):
    def func(functionName):
        def func_in():
            print("----记录日志--args=%s--"%args)
            if args == "heihei":
                functionName()
                functionName()
            else:
                functionName()
        return func_in
    return func

#1.先执行func_arg("heihei")函数,这个函数return 的结果是func这个函数的引用
#2.@func
#3.使用@func对test进行装饰
@func_args("heihei")
def test():
    print("----test----")

#带有参数的装饰器,能够起到在运行时,有不同的功能
@func_args("haha")
def test2():
    print("----test2----")

test()
test2()

----记录日志--args=heihei--
----test----
----test----
----记录日志--args=haha--
----test2----


# 第三章
## 演示顺序

示例1: 使用语法糖@来装饰函数，相当于“myfunc = deco(myfunc)”
但发现新函数只在第一次被调用，且原函数多调用了一次

In [58]:
def deco(func):
    print("before myfunc() called.")
    func()
    print("  after myfunc() called.")
    return func
 
@deco
def myfunc():
    print(" myfunc() called.")
 
myfunc()

before myfunc() called.
 myfunc() called.
  after myfunc() called.
 myfunc() called.


这是一个最简单的装饰器的例子，但是这里有一个问题，就是当我们两次调用myfunc()的时候，发现装饰器函数只被调用了一次。为什么会这样呢？要解释这个就要给出破解装饰器的关键钥匙了。

**这里@deco这一句，和myfunc = deco(myfunc)其实是完全等价的，只不过是换了一种写法而已**

一定要记住上面这句！！！！

好了，从现在开始，只需要做替换操作就可以了。

将@deco 替换为 myfunc = deco(myfunc)

程序首先调用deco(myfunc)，得到的返回结果赋值给了myfunc （注意：在Python中函数名只是个指向函数首地址的函数指针而已）

而deco(myfunc)的返回值就是函数myfunc()的地址

这样其实myfunc 没有变化，也就是说，最后的两次myfunc()函数调用，其实都没有执行到deco()。

有同学就问了，明明打印了deco()函数里面的内容啊，怎么说没有调用到呢。这位同学一看就是没有注意听讲，那一次打印是在@deco 这一句被执行的。大家亲自动手试一下就会发现” myfunc() called.” 这句打印输出了三次。

多的那次就是@deco这里输出的，因为@deco 等价于myfunc = deco(myfunc)，这里已经调用了deco()函数了。

## 确保装饰器被调用
示例2: 使用内嵌包装函数来确保每次新函数都被调用，
内嵌包装函数的形参和返回值与原函数相同，装饰函数返回内嵌包装函数对象.

这里其实不需要我解释了，还是按照第一步中的方法做替换就可以了。还是啰嗦几句吧。。

@deco 替换为 myfunc = deco(myfunc)

程序首先调用deco(myfunc)，得到的返回结果赋值给了myfunc ，

这样myfunc 就变成了指向函数_deco()的指针

以后的myfunc()，其实是调用_deco()

In [65]:
def deco(func):
    def _deco():
        print("before myfunc() called.")
        func()
        print("  after myfunc() called.")
        # 不需要返回func，实际上应返回原函数的返回值
    return _deco
 
@deco
def myfunc():
    print(" myfunc() called.")
    return 'ok'
 
myfunc()
print('')
myfunc()

before myfunc() called.
 myfunc() called.
  after myfunc() called.

before myfunc() called.
 myfunc() called.
  after myfunc() called.


## 对带参数的函数进行装饰
破案过程和第一步、第二步完全一致，不再重复了
示例5: 对带参数的函数进行装饰，

内嵌包装函数的形参和返回值与原函数相同，装饰函数返回内嵌包装函数对象'''

In [68]:
def deco(func):
    def _deco(a, b):
        print("before myfunc() called.")
        ret = func(a, b)
        print("  after myfunc() called. result: %s" % ret)
        return ret
    return _deco
 
@deco
def myfunc(a, b):
    print(" myfunc(%s,%s) called." % (a, b))
    return a + b
 
myfunc(1, 2)
myfunc(3, 4)

before myfunc() called.
 myfunc(1,2) called.
  after myfunc() called. result: 3
before myfunc() called.
 myfunc(3,4) called.
  after myfunc() called. result: 7


7

## 让装饰器带参数
示例7: 在示例4的基础上，让装饰器带参数，

和上一示例相比在外层多了一层包装。

装饰函数名实际上应更有意义些

In [71]:
def deco(arg):
    def _deco(func):
        def __deco():
            print("before %s called [%s]." % (func.__name__, arg))
            func()
            print("  after %s called [%s]." % (func.__name__, arg))
        return __deco
    return _deco
 
@deco("mymodule")
def myfunc():
    print(" myfunc() called.")
 
@deco("module2")
def myfunc2():
    print(" myfunc2() called.")
 
myfunc()
print('')
myfunc2()

before myfunc called [mymodule].
 myfunc() called.
  after myfunc called [mymodule].

before myfunc2 called [module2].
 myfunc2() called.
  after myfunc2 called [module2].


这种带参数的装饰器怎么解释呢。其实是一样的，还是我们的替换操作

@deco(“mymodule”)替换为myfunc = deco(“mymodule”)(myfunc )

注意啊，这里deco后面跟了两个括号。

有同学要问了，这是什么意思？

其实很简单，先执行deco(“mymodule”)，返回结果为_deco

再执行_deco(myfunc)，得到的返回结果为__deco

所以myfunc = __deco

破案！